# Eléments de programmation

Pour mener à bien un calcul algorithmique le nombre d'éléments de langage n'est pas très important et peut se résumer aux 3 syntaxes suivantes

* Le choix conditionnel **if**-**elseif**-**else**-**end**.
* La boucle **for**-**end**.
* La boucle **while**-**end**

# if ... elseif ... else ... end

Un choix simple si le test est vrai (k==1) alors le bloc d'instruction est évalué
<!-- #endregion -->

In [1]:
k = 1
if k == 1
    println("k=1")
end

k=1


en Julia une autre syntaxe est possible avec

In [2]:
k == 1 && println("k=1")

k=1


Le **else** permet de donner un résultat par défaut...

In [3]:
using Random
k = rand(1:2)
if k != 1
    println("k ≠ 1")
else
    println("k=1")
end

k ≠ 1


Une succession de **elseif** permet de choisir parmi plusieurs critères, dans la succession des blocs de **if** et **elseif** le premier qui est "vrai" est évaluer et l'instruction s'arrète.

In [4]:
@show k = rand(1:3)
if k==1
    println("k=1")
elseif k>1
    println("k>1")
else 
    println("k<1")
end

k = rand(1:3) = 2
k>1


# La boucle for

Elle peut se définir à l'aide d'itérateurs ou de tableaux de valeurs les syntaxes "=" , "in" ou "$\in$" sont équivalentes

In [5]:
for i = 1:10
    println(i)
end

1
2
3
4
5
6
7
8
9
10


In [6]:
for i in ["un" 2 im]
    println(typeof(i))
end

String
Int64
Complex{Bool}


In [7]:
typeof.(["un", 2, im])

3-element Vector{DataType}:
 String
 Int64
 Complex{Bool}

In [8]:
eltype(["un" 2 im])

Any

La commande **break** permet de sortir d'une boucle à tout moment

In [9]:
for i = 1:1000
    println(i)
    if i >= 5
       break
    end
end

1
2
3
4
5


La commande **continue** permet elle de courtcircuiter la boucle en court et de passer à la valeur suivante

In [10]:
for i = 1:10
    if i % 3 != 0 # i modulo 3 different de 0
        continue
    end
    println(i)
end

3
6
9


# La boucle while

Tant que le test est "vrai" le bloc est évalué, le test se faisant en entrée de bloc

In [11]:
k = 0
while k<10
    k += 1  # k=k+1
    println(k)
end

1
2
3
4
5
6
7
8
9
10


De même que la boucle **for** les commandes **break** et **continue** sont valables ...

In [12]:
k=0
while k < 1000
    k += 1  # k=k+1
    if k % 5 != 0 # k modulo 5 diffèrent de 0
        continue # retour en début de boucle avec de nouveau un test sur k
    end
    println(k)
    if k > 30
        break
    end
end

5
10
15
20
25
30
35


In [13]:
function syracuse(n)
    k = 0
    while n > 1
        if n % 2 == 0
            n = n ÷ 2
        else
            n = 3n+1
        end
        k += 1
    end
    return k
end

syracuse(10000000000)

124

# Un peu d'optimisation

*Julia* est dit un langage performant, regardons rapidement quelques exemples à faire ou ne pas faire.

## Exemple de préallocation et utilisation de push!

In [14]:
@time let n = 100000
    A = zeros(0) # Tableau vide qui contiendra des Float64
    for i = 1:n
        A = [A; i]   # a chaque itération on change la taille de A
    end
    println(last(A,10))
end

[99991.0, 99992.0, 99993.0, 99994.0, 99995.0, 99996.0, 99997.0, 99998.0, 99999.0, 100000.0]
 19.149154 seconds (2.38 M allocations: 37.313 GiB, 16.20% gc time, 1.61% compilation time)


Cet exemple montre le coût prohibitif d'une réallocation dynamique qui impose une recopie totale de A à chaque itération. Il faut donc récrire notre code différemment pour réduire cet utilisation excessive du ramasse miettes.
Heureusement plusieurs solutions s'offrent à vous

 - Utiliser la fonction `push!`
 - Allouer le tableau `A` avant la boucle si on cannait sa taille
 - Utiliser un itérateur avec la fonction `collect` forsque c'est possible
 - Utiliser la comprehension 

In [15]:
@time let n = 100000
    A = zeros(0)
    for i = 1:n
        push!(A,i)   # a chaque itération on change la taille de A
    end
    println(last(A,10))
end

[99991.0, 99992.0, 99993.0, 99994.0, 99995.0, 99996.0, 99997.0, 99998.0, 99999.0, 100000.0]
  0.001691 seconds (363 allocations: 1.839 MiB)


In [18]:
@time let n = 100000
    A = zeros(Int64,n)
    for i in eachindex(A)
        A[i] = i
    end
    println(last(A,10))
end

[99991, 99992, 99993, 99994, 99995, 99996, 99997, 99998, 99999, 100000]
  0.001620 seconds (327 allocations: 789.047 KiB)


In [19]:
@time let n = 100000
    A = collect(1:n)
    println(last(A,10))
end

[99991, 99992, 99993, 99994, 99995, 99996, 99997, 99998, 99999, 100000]
  0.001031 seconds (325 allocations: 787.453 KiB)


In [20]:
@time let n = 100000
    A = [i for i=1:n]
    println(last(A,10))
end

[99991, 99992, 99993, 99994, 99995, 99996, 99997, 99998, 99999, 100000]
  0.000976 seconds (327 allocations: 789.047 KiB)


## Exemple de vectorisation

Regardons la vectorisation sous *Julia* à l'aide de la construction d'une matrice de Vandermonde

In [26]:
@time let n = 3000
    x = LinRange(0, 1, n)
    V = zeros(n,n)
    for i = axes(V,1)
        V[:,i] .= x.^(i-1) # calcul vectorisé
    end
end

  0.465299 seconds (3 allocations: 68.665 MiB, 13.01% gc time)


In [45]:
@time let n = 5000
    x = LinRange(0, 1, n)
    V = zeros(n,n)
    for i = axes(V,1)
        V[i,:] .= x.^(i-1) # calcul vectorisé
    end
end

  1.329071 seconds (3 allocations: 190.735 MiB, 4.76% gc time)


In [46]:
@time let n = 5000
    x = LinRange(0, 1, n)
    V = zeros(n,n)
    for j=axes(V,2)
        for i=axes(V,1)
            V[i,j] = x[i]^(j-1) # calcul dévectorisé
        end
    end
end

  1.240383 seconds (3 allocations: 190.735 MiB, 1.09% gc time)


In [47]:
@time let n = 5000
    x = LinRange(0, 1, n)
    V = zeros(n,n)
    for i=axes(V,1)
        for j=axes(V,2)
           V[i,j] = x[i]^(j-1) # calcul dévectorisé
        end
    end
end

  1.737419 seconds (3 allocations: 190.735 MiB, 0.70% gc time)


In [39]:
@time let n = 3000
    x = LinRange(0, 1, n)
    W = [x[i]^(j-1) for j=1:n, i=1:n]
end

  0.658236 seconds (110.93 k allocations: 76.306 MiB, 10.54% compilation time)


3000×3000 Matrix{Float64}:
 1.0  1.0          1.0          …  1.0        1.0       1.0       1.0
 0.0  0.000333444  0.000666889     0.999      0.999333  0.999667  1.0
 0.0  1.11185e-7   4.44741e-7      0.998      0.998667  0.999333  1.0
 0.0  3.70741e-11  2.96593e-10     0.997002   0.998001  0.999     1.0
 0.0  1.23622e-14  1.97794e-13     0.996005   0.997335  0.998667  1.0
 0.0  4.12209e-18  1.31907e-16  …  0.995008   0.99667   0.998334  1.0
 0.0  1.37449e-21  8.79673e-20     0.994013   0.996005  0.998001  1.0
 0.0  4.58316e-25  5.86644e-23     0.993019   0.995341  0.997668  1.0
 0.0  1.52823e-28  3.91226e-26     0.992025   0.994677  0.997336  1.0
 0.0  5.09579e-32  2.60905e-29     0.991033   0.994014  0.997003  1.0
 0.0  1.69916e-35  1.73994e-32  …  0.990042   0.993351  0.996671  1.0
 0.0  5.66577e-39  1.16035e-35     0.989051   0.992689  0.996338  1.0
 0.0  1.88922e-42  7.73824e-39     0.988062   0.992027  0.996006  1.0
 ⋮                              ⋱                              

Pour obtenir de meilleures performances il faut le plus souvent possible écrire des fonctions.

In [42]:
using BenchmarkTools

function Vander1(n)
    x = LinRange(0, 1, n)
    V = zeros(n, n)
    for i=1:n
        V[:,i] .= x.^(i-1)
    end
    return V
end
@btime Z = Vander1(3000)

  369.993 ms (2 allocations: 68.66 MiB)


3000×3000 Matrix{Float64}:
 1.0  0.0          0.0         0.0          …  0.0          0.0
 1.0  0.000333444  1.11185e-7  3.70741e-11     0.0          0.0
 1.0  0.000666889  4.44741e-7  2.96593e-10     0.0          0.0
 1.0  0.00100033   1.00067e-6  1.001e-9        0.0          0.0
 1.0  0.00133378   1.77896e-6  2.37274e-9      0.0          0.0
 1.0  0.00166722   2.77963e-6  4.63426e-9   …  0.0          0.0
 1.0  0.00200067   4.00267e-6  8.00801e-9      0.0          0.0
 1.0  0.00233411   5.44808e-6  1.27164e-8      0.0          0.0
 1.0  0.00266756   7.11585e-6  1.89819e-8      0.0          0.0
 1.0  0.003001     9.006e-6    2.7027e-8       0.0          0.0
 1.0  0.00333444   1.11185e-5  3.70741e-8   …  0.0          0.0
 1.0  0.00366789   1.34534e-5  4.93456e-8      0.0          0.0
 1.0  0.00400133   1.60107e-5  6.4064e-8       0.0          0.0
 ⋮                                          ⋱               
 1.0  0.996332     0.992678    0.989037        1.64276e-5   1.63673e-5
 1.0  0.9

In [43]:
@benchmark Z = Vander1(3000)

BenchmarkTools.Trial: 14 samples with 1 evaluation.
 Range (min … max):  352.713 ms … 411.177 ms  ┊ GC (min … max): 1.19% … 3.98%
 Time  (median):     367.749 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   373.594 ms ±  18.128 ms  ┊ GC (mean ± σ):  0.91% ± 1.60%

  ▁  ▁▁    ▁  █  █    ▁ ▁    ▁                ▁         ▁     ▁  
  █▁▁██▁▁▁▁█▁▁█▁▁█▁▁▁▁█▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█ ▁
  353 ms           Histogram: frequency by time          411 ms <

 Memory estimate: 68.66 MiB, allocs estimate: 2.

In [44]:
function Vander2(n)
    x = LinRange(0, 1, n)
    V = zeros(n, n)
    for j=1:n, i=1:n
        V[i,j]=x[i]^(j-1) # calcul dévectorisé
    end
    return V
end
@btime Z = Vander2(3000)

  344.251 ms (2 allocations: 68.66 MiB)


3000×3000 Matrix{Float64}:
 1.0  0.0          0.0         0.0          …  0.0          0.0
 1.0  0.000333444  1.11185e-7  3.70741e-11     0.0          0.0
 1.0  0.000666889  4.44741e-7  2.96593e-10     0.0          0.0
 1.0  0.00100033   1.00067e-6  1.001e-9        0.0          0.0
 1.0  0.00133378   1.77896e-6  2.37274e-9      0.0          0.0
 1.0  0.00166722   2.77963e-6  4.63426e-9   …  0.0          0.0
 1.0  0.00200067   4.00267e-6  8.00801e-9      0.0          0.0
 1.0  0.00233411   5.44808e-6  1.27164e-8      0.0          0.0
 1.0  0.00266756   7.11585e-6  1.89819e-8      0.0          0.0
 1.0  0.003001     9.006e-6    2.7027e-8       0.0          0.0
 1.0  0.00333444   1.11185e-5  3.70741e-8   …  0.0          0.0
 1.0  0.00366789   1.34534e-5  4.93456e-8      0.0          0.0
 1.0  0.00400133   1.60107e-5  6.4064e-8       0.0          0.0
 ⋮                                          ⋱               
 1.0  0.996332     0.992678    0.989037        1.64276e-5   1.63673e-5
 1.0  0.9

In [45]:
function Vander3(n)
    x = LinRange(0, 1, n)
    return [x[i]^(j-1) for i=1:n, j=1:n]
end
@btime Z=Vander3(3000)

  453.929 ms (2 allocations: 68.66 MiB)


3000×3000 Matrix{Float64}:
 1.0  0.0          0.0         0.0          …  0.0          0.0
 1.0  0.000333444  1.11185e-7  3.70741e-11     0.0          0.0
 1.0  0.000666889  4.44741e-7  2.96593e-10     0.0          0.0
 1.0  0.00100033   1.00067e-6  1.001e-9        0.0          0.0
 1.0  0.00133378   1.77896e-6  2.37274e-9      0.0          0.0
 1.0  0.00166722   2.77963e-6  4.63426e-9   …  0.0          0.0
 1.0  0.00200067   4.00267e-6  8.00801e-9      0.0          0.0
 1.0  0.00233411   5.44808e-6  1.27164e-8      0.0          0.0
 1.0  0.00266756   7.11585e-6  1.89819e-8      0.0          0.0
 1.0  0.003001     9.006e-6    2.7027e-8       0.0          0.0
 1.0  0.00333444   1.11185e-5  3.70741e-8   …  0.0          0.0
 1.0  0.00366789   1.34534e-5  4.93456e-8      0.0          0.0
 1.0  0.00400133   1.60107e-5  6.4064e-8       0.0          0.0
 ⋮                                          ⋱               
 1.0  0.996332     0.992678    0.989037        1.64276e-5   1.63673e-5
 1.0  0.9

La conclusion n'est pas toujours évidente a priori... Néanmoins on
peut voir que le fait de mettre le code dans une fonction impose
une optimisation à la compilation et une contextualisation du type
et que l'écriture explicite des boucles donne un meilleur résultat.